In [2]:
pip install h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 288.3 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install tables


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 346.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 248.3 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 340.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.0/162.0 kB 393.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 444.1 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [77]:
import h5py
import pandas as pd
from scipy.sparse import csc_matrix,hstack
import numpy as np




In [78]:
with h5py.File(file_path, 'r') as hdf:
    dataset = hdf['matrix']
    
    df = pd.DataFrame(dataset)
    print(df.head())

          0
0  barcodes
1      data
2  features
3   indices
4    indptr


In [79]:
file_path_1 = 'BRCA_GSE143423_expression.h5'
with h5py.File(file_path_1, 'r') as hdf:
    matrix_group_1 = hdf['matrix']
    
    data_1 = matrix_group_1['data'][:]
    indices_1 = matrix_group_1['indices'][:]
    indptr_1 = matrix_group_1['indptr'][:]
    shape_1 = matrix_group_1['shape'][:]
    
    barcodes_1 = matrix_group_1['barcodes'][:].astype(str)
    features_1 = matrix_group_1['features']['name'][:].astype(str)

file_path_2 = 'BRCA_GSE148673_expression.h5'
with h5py.File(file_path_2, 'r') as hdf:
    matrix_group_2 = hdf['matrix']
    
    data_2 = matrix_group_2['data'][:]
    indices_2 = matrix_group_2['indices'][:]
    indptr_2 = matrix_group_2['indptr'][:]
    shape_2 = matrix_group_2['shape'][:]
    
    barcodes_2 = matrix_group_2['barcodes'][:].astype(str)
    features_2 = matrix_group_2['features']['name'][:].astype(str)

sparse_matrix_1 = csc_matrix((data_1, indices_1, indptr_1), shape=shape_1)
sparse_matrix_2 = csc_matrix((data_2, indices_2, indptr_2), shape=shape_2)

features_1_set = set(features_1)
features_2_set = set(features_2)

common_features = list(features_1_set.intersection(features_2_set))

features_only_in_1 = features_1_set.difference(features_2_set)
features_only_in_2 = features_2_set.difference(features_1_set)

common_indices_1 = [i for i, feature in enumerate(features_1) if feature in common_features]
common_indices_2 = [i for i, feature in enumerate(features_2) if feature in common_features]

sparse_matrix_1_common = sparse_matrix_1[common_indices_1, :]
sparse_matrix_2_common = sparse_matrix_2[common_indices_2, :]

joined_sparse_matrix = hstack([sparse_matrix_1_common, sparse_matrix_2_common])

combined_barcodes = list(barcodes_1) + list(barcodes_2)

df_combined = pd.DataFrame.sparse.from_spmatrix(joined_sparse_matrix, index=common_features, columns=combined_barcodes)

print(df_combined.head())

print(f"Number of common features: {len(common_features)}")
print(f"Number of features only in dataset 1: {len(features_only_in_1)}")
print(f"Number of features only in dataset 2: {len(features_only_in_2)}")

        tnbc1_AAACCTGCACAAGCCC  tnbc1_AAACCTGGTGGGTCAA  \
FBXO22                 0.62607                0.734069   
COL4A2                       0                       0   
PLAGL1                       0                       0   
JAK1                  1.283912                0.432931   
H3F3A                        0                0.432931   

        tnbc1_AAACCTGTCAACGGCC  tnbc1_AAAGATGCATTTCAGG  \
FBXO22                       0                       0   
COL4A2                       0                       0   
PLAGL1                       0                       0   
JAK1                         0                0.646184   
H3F3A                        0                       0   

        tnbc1_AAAGATGGTTAAGGGC  tnbc1_AAAGCAAAGCGTCTAT  \
FBXO22                       0                       0   
COL4A2                       0                       0   
PLAGL1                       0                       0   
JAK1                  0.687111                 0.47606   
H3F3A       

In [80]:
df_cleaned = df_combined.fillna(0)
df_normalized_log = np.log1p(df_cleaned)
print(df_cleaned.head())

        tnbc1_AAACCTGCACAAGCCC  tnbc1_AAACCTGGTGGGTCAA  \
FBXO22                 0.62607                0.734069   
COL4A2                       0                       0   
PLAGL1                       0                       0   
JAK1                  1.283912                0.432931   
H3F3A                        0                0.432931   

        tnbc1_AAACCTGTCAACGGCC  tnbc1_AAAGATGCATTTCAGG  \
FBXO22                       0                       0   
COL4A2                       0                       0   
PLAGL1                       0                       0   
JAK1                         0                0.646184   
H3F3A                        0                       0   

        tnbc1_AAAGATGGTTAAGGGC  tnbc1_AAAGCAAAGCGTCTAT  \
FBXO22                       0                       0   
COL4A2                       0                       0   
PLAGL1                       0                       0   
JAK1                  0.687111                 0.47606   
H3F3A       

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [83]:
print(df_cleaned.columns)


Index(['tnbc1_AAACCTGCACAAGCCC', 'tnbc1_AAACCTGGTGGGTCAA',
       'tnbc1_AAACCTGTCAACGGCC', 'tnbc1_AAAGATGCATTTCAGG',
       'tnbc1_AAAGATGGTTAAGGGC', 'tnbc1_AAAGCAAAGCGTCTAT',
       'tnbc1_AAAGCAACACATTCGA', 'tnbc1_AAAGCAATCGCTAGCG',
       'tnbc1_AAAGTAGCACGAAGCA', 'tnbc1_AAAGTAGGTAGCACGA',
       ...
       'GSM4476489@TTCACGCCAGAAGTGC', 'GSM4476489@TTCTCTCGTCAGTCTA',
       'GSM4476489@TTTCATGTCGGAGTAG', 'GSM4476489@TTTGACTCAAGGTTGG',
       'GSM4476490@ACACTGATCATAGAGA', 'GSM4476490@ACTCTCGAGCGTGTTT',
       'GSM4476490@ATTATCCTCCATTGGA', 'GSM4476490@CTGCGAGTCCCGAACG',
       'GSM4476490@TCAGGGCGTAGTTCCA', 'GSM4476490@TTGGTTTTCATGCTAG'],
      dtype='object', length=14734)


In [84]:

num_samples = df_cleaned.shape[1]

target = [1 if 'tnbc1_' in col else 0 for col in df_cleaned.columns]

target_series = pd.Series(target, index=df_cleaned.columns, name="Target")

print(target_series)


X = df_cleaned.T  
y = target_series

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print(f"F1 Score: {f1}")

tnbc1_AAACCTGCACAAGCCC         1
tnbc1_AAACCTGGTGGGTCAA         1
tnbc1_AAACCTGTCAACGGCC         1
tnbc1_AAAGATGCATTTCAGG         1
tnbc1_AAAGATGGTTAAGGGC         1
                              ..
GSM4476490@ACTCTCGAGCGTGTTT    0
GSM4476490@ATTATCCTCCATTGGA    0
GSM4476490@CTGCGAGTCCCGAACG    0
GSM4476490@TCAGGGCGTAGTTCCA    0
GSM4476490@TTGGTTTTCATGCTAG    0
Name: Target, Length: 14734, dtype: int64
Accuracy: 0.8357837593304682
Confusion Matrix:
[[3637  365]
 [ 361   58]]
F1 Score: 0.1377672209026128
